In [1]:
import warnings
warnings.filterwarnings('ignore')

# import preprocessing function
from helpers.preprocessing import *

# import dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sqlalchemy import create_engine
import psycopg2
from config import db_password

# import machine learning dependencies
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# import neural network dependencies
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Read the Data from SQL

In [2]:
X, y = preprocessing('fall')

# drop beds and bedrooms, as they probably have high colinearity with acommodates
X.drop(['beds', 'bedrooms'], axis = 1, inplace = True)

X.head()

,accommodates,longitude,latitude,review_scores_rating,bathrooms,room_Entire home/apt,room_Hotel room,room_Private room,room_Shared room,property_Camper/RV,property_Entire Home,property_Entire Unit,property_Other,property_Private room,property_Shared room
0,8,-117.25760,32.80751,4.76,2.0,1,0,0,0,0,1,0,0,0,0
1,1,-117.21931,32.74217,4.33,1.0,0,0,1,0,0,0,0,0,1,0
2,7,-117.25416,32.79783,4.75,2.5,1,0,0,0,0,1,0,0,0,0
3,8,-117.25728,32.80751,4.72,2.0,1,0,0,0,0,1,0,0,0,0
4,3,-117.26856,32.81301,4.94,1.0,1,0,0,0,0,1,0,0,0,0


# Split the Data into Training and Testing

In [3]:
X.describe()

,accommodates,longitude,latitude,review_scores_rating,bathrooms,room_Entire home/apt,room_Hotel room,room_Private room,room_Shared room,property_Camper/RV,property_Entire Home,property_Entire Unit,property_Other,property_Private room,property_Shared room
count,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000
mean,4.866391,-117.173776,32.763467,4.749504,1.603629,0.838372,0.001069,0.155497,0.005062,0.005989,0.406459,0.422644,0.003066,0.156780,0.005062
std,3.133007,0.074927,0.096004,0.387233,0.984302,0.368123,0.032686,0.362391,0.070970,0.077159,0.491190,0.493997,0.055286,0.363606,0.070970
min,1.000000,-117.282580,32.519860,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,-117.248773,32.719043,4.710000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,-117.166917,32.756855,4.820000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6.000000,-117.129102,32.798990,4.950000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,16.000000,-116.896560,33.125690,5.000000,27.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
# split data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [5]:
# scale the X data
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Multiple Linear Regression Model

In [6]:
reg = LinearRegression()
reg.fit(X_train_scaled, y_train)
y_pred = reg.predict(X_test_scaled)

In [7]:
print(f'Training R^2: {reg.score(X_train_scaled, y_train)}')
print(f'Testing R^2: {reg.score(X_test_scaled, y_test)}')

Training R^2: 0.09456832376223434
Testing R^2: 0.014471680833533451


In [8]:
# Display the coefficients in the model
results_df = pd.DataFrame({'Values': X.columns, 'Coef':reg.coef_})
results_df

,Values,Coef
0,accommodates,8.545343e+01
1,longitude,-4.178743e+01
2,latitude,2.870760e+01
3,review_scores_rating,-1.696696e+01
4,bathrooms,1.600779e+02
5,room_Entire home/apt,-2.099757e+15
6,room_Hotel room,-2.001754e+14
7,room_Private room,-2.064866e+15
8,room_Shared room,2.550783e+14
9,property_Camper/RV,8.772147e+14


## Neural Network

In [9]:
X_train_scaled.shape

(10519, 15)

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train_scaled.shape[1]
hidden_nodes_layer1 = 63
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 63)                1008      
                                                                 
 dense_1 (Dense)             (None, 30)                1920      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 2,959
Trainable params: 2,959
Non-trainable params: 0
_________________________________________________________________


In [11]:
nn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [12]:
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
329/329 [==============================] - 1s 932us/step - loss: 163.9281
Epoch 2/100
329/329 [==============================] - 0s 954us/step - loss: 157.0298
Epoch 3/100
329/329 [==============================] - 0s 876us/step - loss: 154.4086
Epoch 4/100
329/329 [==============================] - 0s 837us/step - loss: 153.3988
Epoch 5/100
329/329 [==============================] - 0s 902us/step - loss: 153.8130
Epoch 6/100
329/329 [==============================] - 0s 883us/step - loss: 152.1198
Epoch 7/100
329/329 [==============================] - 0s 908us/step - loss: 153.2334
Epoch 8/100
329/329 [==============================] - 0s 1ms/step - loss: 152.3027
Epoch 9/100
329/329 [==============================] - 1s 2ms/step - loss: 152.4430
Epoch 10/100
329/329 [==============================] - 0s 2ms/step - loss: 152.5925
Epoch 11/100
329/329 [==============================] - 1s 2ms/step - loss: 151.2524
Epoch 12/100
329/329 [==============================] - 1s 2

329/329 [==============================] - 0s 921us/step - loss: 148.6965
Epoch 98/100
329/329 [==============================] - 0s 910us/step - loss: 149.1202
Epoch 99/100
329/329 [==============================] - 0s 1ms/step - loss: 148.3772
Epoch 100/100
329/329 [==============================] - 0s 1ms/step - loss: 148.2137


In [13]:
# Evaluate the mean absolute error of the  model using the test data
mae = nn.evaluate(X_test_scaled,y_test,verbose=0)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 179.67294311523438
